# 导入工具包

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import riiideducation
env = riiideducation.make_env()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 数据地址

In [2]:
dir_path = '/kaggle/input/riiid-test-answer-prediction/'
file_train = 'train.csv'
file_questions = 'questions.csv'
file_lectures = 'lectures.csv'

# 数据大小

In [3]:
# nrows =  5 * 100 * 10000
nrows = None

# 读取训练数据

In [4]:
train = pd.read_csv(
                    dir_path + file_train, 
                    nrows=nrows, 
                    usecols=['row_id', 'timestamp', 'user_id', 'content_id', 
                             'content_type_id', 'task_container_id', 'answered_correctly',
                            'prior_question_elapsed_time','prior_question_had_explanation'],
                    dtype={
                            'row_id': 'int64',
                            'timestamp': 'int64',
                            'user_id': 'int32',
                            'content_id': 'int16',
                            'content_type_id': 'int8',
                            'task_container_id': 'int8',
                            'answered_correctly': 'int8',
                            'prior_question_elapsed_time': 'float32',
                            'prior_question_had_explanation': 'str'
                        }
                   )

# 读取问题数据

In [5]:
questions = pd.read_csv(
                        dir_path + file_questions, 
                        nrows=nrows,
                        usecols=['question_id','bundle_id','part'], 
                        dtype={
                           'question_id': 'int16',
                           'bundle_id': 'int16',
                           'part': 'int8',
                       }
                    )

# 读取讲课数据

In [6]:
lectures = pd.read_csv(
                       dir_path + file_lectures, 
                       usecols=['lecture_id','tag','part','type_of'], 
                       nrows=nrows,
                       dtype={
                           'lecture_id': 'int16',
                           'tag': 'int16',
                           'part': 'int8',
                           'type_of': 'str'
                       }
                    )

# prior_question_had_explanation 特征处理

In [7]:
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].map({'True':1,'False':0}).fillna(-1).astype(np.int8)

# 只保留问题数据

In [8]:
train = train[train['content_type_id']==0]

# 释放内存

In [9]:
import gc

In [10]:
gc.collect()

62

# tailf选取部分数据进行建模，减少内存压力

In [11]:
# 压缩内存
max_num = 200
train = train.groupby(['user_id']).tail(max_num)

# 训练数据和问题数据进行关联

In [12]:
train = pd.merge(
        left=train,
        right=questions,
        how='left',
        left_on='content_id',
        right_on='question_id'
        )

# 训练数据和讲座数据进行关联

In [13]:
# train = pd.merge(
#         left=train,
#         right=lectures,
#         how='right',
#         left_on='content_id',
#         right_on='lecture_id'
#         )

# 训练数据填充0

In [14]:
train = train.fillna(0)

In [15]:
print(train)

             row_id  timestamp     user_id  content_id  content_type_id  \
0                 0          0         115        5692                0   
1                 1      56943         115        5716                0   
2                 2     118363         115         128                0   
3                 3     131167         115        7860                0   
4                 4     137965         115        7922                0   
...             ...        ...         ...         ...              ...   
32215867  101230327  428564420  2147482888        3586                0   
32215868  101230328  428585000  2147482888        6341                0   
32215869  101230329  428613475  2147482888        4212                0   
32215870  101230330  428649406  2147482888        6343                0   
32215871  101230331  428692118  2147482888        7995                0   

          task_container_id  answered_correctly  prior_question_elapsed_time  \
0                  

# 类别特征转换函数

In [16]:
class cat_deal:
    def __init__(self):
        self.max_len = 0
        self.dict_map = {}
    
    def fit(self, cat_list):
        index = 1 
        for cat_i in cat_list:
            if cat_i not in self.dict_map:
                self.dict_map[cat_i] = index
                index += 1
        self.max_len = index + 1
        
    def transform(self, cat_list):
        cat_transform_list = []
        for cat_i in cat_list:
            if cat_i in self.dict_map:
                cat_transform_list.append(self.dict_map[cat_i])
            else:
                cat_transform_list.append(0)
        return cat_transform_list

# 浮点特征转换函数

In [17]:
class float_deal:
    def __init__(self):
        self.max = 0
        self.min = 0
        self.max_min = 0 
        
    def fit(self, float_list):
        for float_i in float_list:
            if float_i < self.min:
                self.min = float_i
            if float_i > self.max:
                self.max = float_i
        self.max_min = self.max - self.min
        
    def transform(self, float_list):
        float_transform_list = []
        for float_i in float_list:
            if float_i < self.min:
                float_transform_list.append(0)
            elif float_i > self.max:
                float_transform_list.append(1)
            else:
                float_transform_list.append(float_i/self.max_min)
        return float_transform_list

# 处理类别特征

In [18]:
dict_cat_class = {}
for columns in ['user_id','content_id',\
                'task_container_id','prior_question_had_explanation',\
                'bundle_id','part']:
    dict_cat_class[columns] = cat_deal()
    dict_cat_class[columns].fit(train[columns])

    train[columns] = dict_cat_class[columns].transform(train[columns])
    print(columns)

user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part


# 处理浮点特征

In [19]:
dict_float_class = {}
for columns in ['timestamp','prior_question_elapsed_time']:
    dict_float_class[columns] = float_deal()
    dict_float_class[columns].fit(train[columns])
    
    train[columns] = dict_float_class[columns].transform(train[columns])
    print(columns)

timestamp
prior_question_elapsed_time


# 模型使用的函数（keras Lambda函数包装）

In [20]:
def squeeze(embedding):
    embedding = tf.squeeze(embedding,axis=1)
    return embedding
def concat(embedding_list):
    embedding = tf.concat(embedding_list, axis=1)
    return embedding
def multiply(multi_x_y):
    multi_x = multi_x_y[0]
    multi_y = multi_x_y[1]
    multi_x_y = tf.multiply(multi_x, multi_y)
    return multi_x_y

In [21]:
# tf.squeeze?

# 建立模型和编译

In [22]:
input_timestamp = tf.keras.Input(shape=(1,))
input_prior_question_elapsed_time = tf.keras.Input(shape=(1,))

# input int
input_user = tf.keras.Input(shape=(1,))
input_content = tf.keras.Input(shape=(1,))
input_task_container = tf.keras.Input(shape=(1,))
input_prior_question_had_explanation = tf.keras.Input(shape=(1,))
input_bundle = tf.keras.Input(shape=(1,))
input_part = tf.keras.Input(shape=(1,))
# input_question = tf.keras.Input(shape=(1,))

inputs = [input_timestamp,input_prior_question_elapsed_time,\
         input_user,input_content,\
         input_task_container,input_prior_question_had_explanation,\
         input_bundle,input_part]
# inputs = tf.keras.layers.Lambda(concat)(inputs)

# input session
# input_tags = Input(shape=(1))

# embedding float
embedding_timestamp = tf.keras.layers.Dense(64, activation=tf.nn.sigmoid)(input_timestamp)
embedding_prior_question_elapsed_time = tf.keras.layers.Dense(64, activation=tf.nn.sigmoid)(input_prior_question_elapsed_time)

# embedding int 
embedding_user = tf.keras.layers.Embedding(dict_cat_class['user_id'].max_len,
                                           64, input_length=1)(input_user)
embedding_user = tf.keras.layers.Lambda(squeeze)(embedding_user)

embedding_content = tf.keras.layers.Embedding(dict_cat_class['content_id'].max_len,
                                              64, input_length=1)(input_content)
embedding_content = tf.keras.layers.Lambda(squeeze)(embedding_content)

embedding_task_container = tf.keras.layers.Embedding(dict_cat_class['task_container_id'].max_len,
                                                     64, input_length=1)(input_task_container)
embedding_task_container = tf.keras.layers.Lambda(squeeze)(embedding_task_container)

embedding_prior_question_had_explanation = tf.keras.layers.Embedding(dict_cat_class['prior_question_had_explanation'].max_len, 
                                                                     64, input_length=1)(input_prior_question_had_explanation)
embedding_prior_question_had_explanation = tf.keras.layers.Lambda(squeeze)(embedding_prior_question_had_explanation)

embedding_bundle = tf.keras.layers.Embedding(dict_cat_class['bundle_id'].max_len,
                                             64, input_length=1)(input_bundle)
embedding_bundle = tf.keras.layers.Lambda(squeeze)(embedding_bundle)

embedding_part = tf.keras.layers.Embedding(dict_cat_class['part'].max_len,
                                           64, input_length=1)(input_part)
embedding_part = tf.keras.layers.Lambda(squeeze)(embedding_part)

# embedding_question = tf.keras.layers.Embedding(dict_cat_class['question_id'].max_len,
#                                            64, input_length=1)(input_question)
# embedding_question = tf.keras.layers.Lambda(squeeze)(embedding_question)

embedding_all = [embedding_timestamp,embedding_prior_question_elapsed_time,\
                embedding_user, embedding_content, embedding_task_container,\
                embedding_prior_question_had_explanation, embedding_bundle, embedding_part]


nffm1, nffm2 = [], []
for i, embedding_i in enumerate(embedding_all):
    for j, embedding_j in enumerate(embedding_all):
        if i > j:
            nffm1.append(embedding_i), nffm2.append(embedding_j)
nffm1_layer = tf.keras.layers.Lambda(concat)(nffm1)
nffm2_layer = tf.keras.layers.Lambda(concat)(nffm2)     

nffm_all = tf.keras.layers.Lambda(multiply)([nffm1_layer,nffm2_layer])
    
logit = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(nffm_all)


model = tf.keras.models.Model(inputs=inputs, outputs=logit)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_crossentropy'])

In [23]:
nffm1, nffm2 = [], []
for i, embedding_i in enumerate(embedding_all):
    for j, embedding_j in enumerate(embedding_all):
        if i > j:
            nffm1.append(embedding_i), nffm2.append(embedding_j)
nffm1_layer = tf.keras.layers.Lambda(concat)(nffm1)
nffm2_layer = tf.keras.layers.Lambda(concat)(nffm2) 

nffm_all = tf.keras.layers.Lambda(multiply)([nffm1_layer,nffm2_layer])

In [24]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1)]          0                                            
_______________________________________________________________________________________

# 模型训练

In [25]:
plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                            verbose=0,
                            mode='min',
                            factor=0.1,
                            patience=6)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                               verbose=0,
                               mode='min',
                               patience=10)

# 保存
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'fold.h5',
                             monitor='val_loss',
                             verbose=0,
                             mode='min',
                             save_best_only=True)

# 验证集和训练集初始数据获取

In [26]:
valid = pd.DataFrame()
for i in range(10):
    
    # 获取训练标签数据
    last_records = train.drop_duplicates('user_id', keep='last')
    
    # 获取训练标签以前的数据
    map__last_records__user_row = dict(zip(last_records['user_id'],last_records['row_id']))
    train['filter_row'] = train['user_id'].map(map__last_records__user_row)
    train = train[train['row_id']<train['filter_row']]

    # 特征加入训练集
    valid = valid.append(last_records)
    print(len(valid))

393656
787225
1180734
1574202
1967602
2360984
2754348
3146047
3537383
3928411


# 获取训练和验证数据

In [27]:
features_columns = ['timestamp','prior_question_elapsed_time',\
                    'user_id','content_id',\
                    'task_container_id','prior_question_had_explanation',\
                    'bundle_id','part']

X_test, y_test = [valid[columns].values for columns in features_columns], valid['answered_correctly'].values
# del valid

X_train, y_train = [train[columns].values for columns in features_columns], train['answered_correctly'].values
# del train

In [28]:
print('ok')

ok


# 训练模型

In [29]:
model.fit(X_train, y_train,
          epochs=40,
          batch_size=512 * 500 * 2,
          verbose=1,
          shuffle=True,
          validation_data=(X_test, y_test),
          callbacks=[plateau, early_stopping, checkpoint])

y_test_proba = model.predict(X_test, verbose=0, batch_size=512 * 500)
auc = roc_auc_score(y_test, y_test_proba)
print(auc)


Epoch 1/40
56/56 [==============================] - 68s 1s/step - loss: 0.6465 - binary_crossentropy: 0.6465 - val_loss: 0.6433 - val_binary_crossentropy: 0.6433
Epoch 2/40
56/56 [==============================] - 68s 1s/step - loss: 0.5966 - binary_crossentropy: 0.5966 - val_loss: 0.6237 - val_binary_crossentropy: 0.6237
Epoch 3/40
56/56 [==============================] - 68s 1s/step - loss: 0.5735 - binary_crossentropy: 0.5735 - val_loss: 0.6125 - val_binary_crossentropy: 0.6125
Epoch 4/40
56/56 [==============================] - 67s 1s/step - loss: 0.5583 - binary_crossentropy: 0.5583 - val_loss: 0.6057 - val_binary_crossentropy: 0.6057
Epoch 5/40
56/56 [==============================] - 68s 1s/step - loss: 0.5530 - binary_crossentropy: 0.5530 - val_loss: 0.6023 - val_binary_crossentropy: 0.6023
Epoch 6/40
56/56 [==============================] - 66s 1s/step - loss: 0.5506 - binary_crossentropy: 0.5506 - val_loss: 0.6004 - val_binary_crossentropy: 0.6004
Epoch 7/40
56/56 [==========

In [30]:
model.fit(X_train, y_train,
          epochs=10,
          batch_size=512 * 500 * 2 ,
          verbose=1,
          shuffle=True,
          validation_data=(X_test, y_test),
          callbacks=[plateau, early_stopping, checkpoint])

y_test_proba = model.predict(X_test, verbose=0, batch_size=512 * 500)
auc = roc_auc_score(y_test, y_test_proba)
print(auc)

Epoch 1/10
56/56 [==============================] - 66s 1s/step - loss: 0.5370 - binary_crossentropy: 0.5370 - val_loss: 0.6009 - val_binary_crossentropy: 0.6009
Epoch 2/10
56/56 [==============================] - 66s 1s/step - loss: 0.5367 - binary_crossentropy: 0.5367 - val_loss: 0.6010 - val_binary_crossentropy: 0.6010
Epoch 3/10
56/56 [==============================] - 66s 1s/step - loss: 0.5364 - binary_crossentropy: 0.5364 - val_loss: 0.6010 - val_binary_crossentropy: 0.6010
Epoch 4/10
56/56 [==============================] - 65s 1s/step - loss: 0.5361 - binary_crossentropy: 0.5361 - val_loss: 0.6010 - val_binary_crossentropy: 0.6010
Epoch 5/10
56/56 [==============================] - 65s 1s/step - loss: 0.5358 - binary_crossentropy: 0.5358 - val_loss: 0.6011 - val_binary_crossentropy: 0.6011
Epoch 6/10
56/56 [==============================] - 65s 1s/step - loss: 0.5354 - binary_crossentropy: 0.5354 - val_loss: 0.6012 - val_binary_crossentropy: 0.6012
Epoch 7/10
56/56 [==========

In [31]:
print('ok')

ok


# 提交结果

In [32]:
iter_test = env.iter_test()

for (test_df, sample_prediction_df) in iter_test:

    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].map({'True':1,'False':0}).fillna(-1).astype(np.int8)

    test_df = pd.merge(
        left=test_df,
        right=questions,
        how='left',
        left_on='content_id',
        right_on='question_id'
        )
    
#     test_df = pd.merge(
#         left=test_df,
#         right=lectures,
#         how='right',
#         left_on='content_id',
#         right_on='lecture_id'
#         )

    test_df = test_df.fillna(0)


    for columns in ['user_id','content_id',\
                    'task_container_id','prior_question_had_explanation',\
                    'bundle_id','part']:

        test_df[columns] = dict_cat_class[columns].transform(test_df[columns])
        print(columns)


    for columns in ['timestamp','prior_question_elapsed_time']:

        test_df[columns] = dict_float_class[columns].transform(test_df[columns])
        print(columns)

    X_test = [test_df[columns].values for columns in features_columns]

    test_df['answered_correctly'] =  model.predict(X_test, verbose=0, batch_size=512 * 500)
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])


user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
user_id
content_id
task_container_id
prior_question_had_explanation
bundle_id
part
timestamp
prior_question_elapsed_time
